<a href="https://colab.research.google.com/github/maicken/INF442_P8/blob/master/INF442_P8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data analysis
From the clean data, we will be performing an analysis to find an insight and thus be able to give a suggestion to reduce the number of accidents



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

In [0]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Getting the dataframe that we will use to analyse
%cd "gdrive/My Drive/Colab Notebooks/INF442_P8"

/content/gdrive/My Drive/Colab Notebooks/INF442_P8


In [0]:
df = pd.read_csv('dataframe.csv', index_col=0)
df.info()
df.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232233 entries, 201800000001 to 201000069386
Data columns (total 29 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   num_veh  1232233 non-null  object 
 1   place    1232233 non-null  float64
 2   catu     1232233 non-null  int64  
 3   grav     1232233 non-null  int64  
 4   age      1232233 non-null  float64
 5   trajet   1232233 non-null  float64
 6   secu     1232233 non-null  float64
 7   locp     1232233 non-null  float64
 8   actp     1232233 non-null  float64
 9   etatp    1232233 non-null  float64
 10  an       1232233 non-null  int64  
 11  lum      1232233 non-null  int64  
 12  dep      1232233 non-null  int64  
 13  agg      1232233 non-null  int64  
 14  int      1232233 non-null  int64  
 15  atm      1232233 non-null  float64
 16  col      1232233 non-null  float64
 17  catr     1232233 non-null  int64  
 18  circ     1232233 non-null  float64
 19  nbv      1232233 non-null 

,num_veh,place,catu,grav,age,trajet,secu,locp,actp,etatp,an,lum,dep,agg,int,atm,col,catr,circ,nbv,vosp,plan,surf,infra,situ,catv,obs,choc,manv
num_acc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201800000001,B01,1.0,1,3,90.0,0.0,11.0,0.0,0.0,0.0,18,1,590,1,4,1.0,1.0,3,2.0,2.0,0.0,3.0,1.0,0.0,1.0,3,2.0,3.0,1.0
201800000001,A01,1.0,1,1,58.0,5.0,11.0,0.0,0.0,0.0,18,1,590,1,4,1.0,1.0,3,2.0,2.0,0.0,3.0,1.0,0.0,1.0,3,2.0,2.0,15.0
201800000002,A01,1.0,1,1,71.0,0.0,11.0,0.0,0.0,0.0,18,1,590,2,7,7.0,7.0,4,2.0,2.0,0.0,2.0,1.0,0.0,1.0,3,1.0,0.0,1.0
201800000002,A01,1.0,3,4,59.0,0.0,2.0,2.0,3.0,1.0,18,1,590,2,7,7.0,7.0,4,2.0,2.0,0.0,2.0,1.0,0.0,1.0,3,1.0,0.0,1.0
201800000003,A01,1.0,1,3,31.0,5.0,21.0,0.0,0.0,0.0,18,1,590,2,3,1.0,7.0,3,2.0,2.0,0.0,1.0,1.0,0.0,1.0,2,2.0,1.0,1.0


In [0]:
X_train = df.drop(['num_veh', 'grav'], axis=1)
y_train = df['grav']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

KeyboardInterrupt: ignored

In [0]:
# Columns that we will be using
df_gravity = df.drop(['num_veh', "dep", "an", "locp", "actp", "etatp", "grav", "place", "manv", "plan", "situ", "infra", "atm", "secu"], axis=1)
df_gravity.info()
print('Before dummification we have {} variables in train'.format(len(df_gravity.columns)))
df_gravity = pd.get_dummies(df_gravity, columns=["catv", "choc", "col", "obs", "trajet", "catr", "lum", "catu", "int", "circ", "surf", "agg", "vosp"])
print('After dummification we have {} variables in train'.format(len(df_gravity.columns)))
df_gravity['age'] = df_gravity['age'] / df_gravity['age'].max()
df_gravity['nbv'] = df_gravity['nbv'] / df_gravity['nbv'].max()

X = df_gravity.to_numpy()
y = df['grav'].to_numpy()

# Correct gravity order
# 1 - Indemne 
# 5 - Blessé léger 
# 10 - Blessé hospitalisé 
# 15 - Tué

y = np.where(y==2, 15, y)
y = np.where(y==3, 10, y)
y = np.where(y==4,  5, y)

df_gravity.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1232233 entries, 201800000001 to 201000069386
Data columns (total 15 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   catu    1232233 non-null  int64  
 1   age     1232233 non-null  float64
 2   trajet  1232233 non-null  float64
 3   lum     1232233 non-null  int64  
 4   agg     1232233 non-null  int64  
 5   int     1232233 non-null  int64  
 6   col     1232233 non-null  float64
 7   catr    1232233 non-null  int64  
 8   circ    1232233 non-null  float64
 9   nbv     1232233 non-null  float64
 10  vosp    1232233 non-null  float64
 11  surf    1232233 non-null  float64
 12  catv    1232233 non-null  int64  
 13  obs     1232233 non-null  float64
 14  choc    1232233 non-null  float64
dtypes: float64(9), int64(6)
memory usage: 150.4 MB
Before dummification we have 15 variables in train
After dummification we have 96 variables in train
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12322

In [0]:
print(df["catv"].value_counts())
print(df["obs"].value_counts())
print()
print(df_gravity["catv_1"].value_counts())
print(df_gravity["catv_2"].value_counts())
print(df_gravity["catv_3"].value_counts())
print(df_gravity["catv_4"].value_counts())
print(df_gravity["catv_5"].value_counts())

3    876444
2    248761
4     48672
1     44723
5     13633
Name: catv, dtype: int64
2.0     775367
1.0     221186
0.0     114862
13.0     22101
9.0      15801
6.0      15369
4.0      13758
3.0      11252
8.0      10882
16.0      6942
14.0      6429
12.0      5491
15.0      4528
5.0       2489
11.0      2449
7.0       2055
10.0      1272
Name: obs, dtype: int64

0    1187510
1      44723
Name: catv_1, dtype: int64
0    983472
1    248761
Name: catv_2, dtype: int64
1    876444
0    355789
Name: catv_3, dtype: int64
0    1183561
1      48672
Name: catv_4, dtype: int64
0    1218600
1      13633
Name: catv_5, dtype: int64


In [0]:
import statsmodels.api as sm

model = sm.OLS(X, y)  # model
results = model.fit()
beta = results.params

In [0]:
df_v = pd.DataFrame({"label": df_gravity.columns, "value": beta[0]})
df_v.to_csv("test.csv", sep=";", decimal=",", index=False)

# Data cleaning

Now that we have all the .csv that will be using, we clean those data and join them together

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [0]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Getting the data
%cd "gdrive/My Drive/Colab Notebooks/INF442_P8"

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/INF442_P8'
/content/gdrive/My Drive/Colab Notebooks/INF442_P8


In [0]:
df_c = pd.read_csv('caracteristiques.csv')
df_l = pd.read_csv('lieux.csv')
df_u = pd.read_csv('usagers.csv')
df_v = pd.read_csv('vehicules.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print("Caracteristiques")
print(df_c.head())
print(df_c.columns)
print()
print("Lieux")
print(df_l.head())
print(df_l.columns)
print()
print("Usagers")
print(df_u.head())
print(df_u.columns)
print()
print("Vehicules")
print(df_v.head())
print(df_v.columns)

Caracteristiques
        Num_Acc  an  mois  jour  ...  gps        lat      long  dep
0  201800000001  18     1    24  ...    M  5055737.0  294992.0  590
1  201800000002  18     2    12  ...    M  5052936.0  293151.0  590
2  201800000003  18     3     4  ...    M  5051243.0  291714.0  590
3  201800000004  18     5     5  ...    M  5051974.0  289123.0  590
4  201800000005  18     6    26  ...    M  5051607.0  290605.0  590

[5 rows x 16 columns]
Index(['Num_Acc', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm',
       'col', 'com', 'adr', 'gps', 'lat', 'long', 'dep'],
      dtype='object')

Lieux
        Num_Acc  catr voie  v1   v2  ...  larrout  surf  infra  situ  env1
0  201800000001     3   41 NaN    C  ...      NaN   1.0    0.0   1.0   0.0
1  201800000002     4   41 NaN    D  ...      NaN   1.0    0.0   1.0   0.0
2  201800000003     3   39 NaN    D  ...      NaN   1.0    0.0   1.0   0.0
3  201800000004     3   39 NaN  NaN  ...      NaN   1.0    0.0   1.0   0.0
4  20180000000

In [0]:
# Information about the columns
print("caracteristiques")
print()
df_c.info()
print()
print("lieux")
print()
df_l.info()
print()
print("usagers")
print()
df_u.info()
print()
print("vehicules")
print()
df_v.info()
print()

caracteristiques

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Num_Acc  553424 non-null  int64  
 1   an       553424 non-null  int64  
 2   mois     553424 non-null  int64  
 3   jour     553424 non-null  int64  
 4   hrmn     553424 non-null  int64  
 5   lum      553424 non-null  int64  
 6   agg      553424 non-null  int64  
 7   int      553424 non-null  int64  
 8   atm      553358 non-null  float64
 9   col      553405 non-null  float64
 10  com      553424 non-null  int64  
 11  adr      482138 non-null  object 
 12  gps      349704 non-null  object 
 13  lat      341505 non-null  float64
 14  long     341501 non-null  float64
 15  dep      553424 non-null  int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 67.6+ MB

lieux

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total

In [0]:
# Some histogramas to analyse caracteristiques
print()
print(df_c['dep'].value_counts())
print()
print(df_c['com'].value_counts())
print()
print(df_c['agg'].value_counts())
print()
print(df_c['int'].value_counts())
print()
print(df_c['atm'].value_counts())
print()
print(df_c['col'].value_counts())
print()
print(df_c['gps'].value_counts())


750    58834
130    35762
930    25492
920    23641
940    22648
       ...  
460      854
90       829
150      786
480      587
230      562
Name: dep, Length: 101, dtype: int64

55     23081
88      6919
113     6743
116     6633
7       6543
       ...  
879        1
840        1
881        1
772        1
845        1
Name: com, Length: 901, dtype: int64

2    371614
1    181810
Name: agg, dtype: int64

1    386176
2     66733
3     50705
6     17171
9      9349
4      9104
5      7105
7      6325
8       666
0        90
Name: int, dtype: int64

1.0    446181
2.0     58692
8.0     17569
3.0     12476
7.0      6489
5.0      3702
9.0      3672
4.0      3241
6.0      1336
Name: atm, dtype: int64

6.0    187189
3.0    154642
2.0     67015
7.0     55425
1.0     52682
4.0     18630
5.0     17822
Name: col, dtype: int64

M    336961
A      5127
R      4027
G      2743
Y       842
T         2
S         2
Name: gps, dtype: int64


In [0]:
# Some histogramas to analyse lieux
print()
print(df_l['catr'].value_counts())
print()
print(df_l['circ'].value_counts())
print()
print(df_l['nbv'].value_counts())
print()
print(df_l['vosp'].value_counts())
print()
print(df_l['prof'].value_counts())
print()
print(df_l['plan'].value_counts())
print()
print(df_l['surf'].value_counts())
print()
print(df_l['infra'].value_counts())
print()
print(df_l['situ'].value_counts())
print()
print(df_l['env1'].value_counts())


4    275408
3    184584
1     45562
2     35705
9      7077
6      3759
5       935
7       394
Name: catr, dtype: int64

2.0    340973
1.0    102585
3.0     74068
0.0     31271
4.0      3014
Name: circ, dtype: int64

2.0     308897
1.0      69788
0.0      59199
4.0      51607
3.0      42853
6.0      10257
5.0       5553
8.0       1954
7.0        643
10.0       316
9.0        166
11.0        51
12.0        37
13.0        14
Name: nbv, dtype: int64

0.0    515659
3.0     15770
1.0     12008
2.0      7484
Name: vosp, dtype: int64

1.0    419690
2.0     78928
0.0     36394
3.0      9617
4.0      6925
Name: prof, dtype: int64

1.0    417357
2.0     45182
0.0     41346
3.0     40583
4.0      6750
Name: plan, dtype: int64

1.0    427628
2.0     95148
0.0     18391
7.0      3174
9.0      2797
5.0      1589
8.0      1243
3.0       936
4.0       354
6.0       308
Name: surf, dtype: int64

0.0    488629
5.0     31064
3.0      8601
2.0      7976
1.0      5402
6.0      4104
4.0      2067
7.0     

In [0]:
# Join caracteristiques and lieux into one pandas only
df = df_c.merge(df_l, left_index=True, right_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553424 entries, 0 to 553423
Data columns (total 34 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Num_Acc_x  553424 non-null  int64  
 1   an         553424 non-null  int64  
 2   mois       553424 non-null  int64  
 3   jour       553424 non-null  int64  
 4   hrmn       553424 non-null  int64  
 5   lum        553424 non-null  int64  
 6   agg        553424 non-null  int64  
 7   int        553424 non-null  int64  
 8   atm        553358 non-null  float64
 9   col        553405 non-null  float64
 10  com        553424 non-null  int64  
 11  adr        482138 non-null  object 
 12  gps        349704 non-null  object 
 13  lat        341505 non-null  float64
 14  long       341501 non-null  float64
 15  dep        553424 non-null  int64  
 16  Num_Acc_y  553424 non-null  int64  
 17  catr       553424 non-null  int64  
 18  voie       484820 non-null  object 
 19  v1         2674 non-nul

In [0]:
# Some histogramas to analyse vehicules
print()
print(df_v['senc'].value_counts())
print()
print(df_v['catv'].value_counts())
print()
print(df_v['obs'].value_counts())
print()
print(df_v['obsm'].value_counts())
print()
print(df_v['choc'].value_counts())
print()
print(df_v['manv'].value_counts())
print()


0.0    681840
1.0    159382
2.0    100728
Name: senc, dtype: int64

7     583688
33     77985
10     51847
1      42174
30     40481
2      37307
31     22369
32     21881
34     13542
14      9269
15      8560
37      7163
17      6641
3       3911
13      3632
99      3586
21      1886
38      1871
40      1300
36      1288
20       912
16       400
35       273
39       256
Name: catv, dtype: int64

0.0     819484
1.0      19908
13.0     15165
2.0      13324
4.0      11350
3.0      10419
6.0      10073
8.0       9771
14.0      6660
16.0      5593
12.0      5388
15.0      4523
9.0       3184
11.0      2187
7.0       2122
5.0       1287
10.0      1041
Name: obs, dtype: int64

2.0    635926
0.0    188814
1.0    101842
9.0     11456
6.0      1824
4.0       907
5.0       800
Name: obsm, dtype: int64

1.0    341632
3.0    133996
2.0    108755
4.0     92190
8.0     68541
0.0     64697
7.0     59018
6.0     32166
5.0     25531
9.0     15347
Name: choc, dtype: int64

1.0     402692
2.0     

In [0]:
# Some histogramas to analyse usagers
print()
print(df_u['place'].value_counts())
print()
print(df_u['catu'].value_counts())
print()
print(df_u['grav'].value_counts())
print()
print(df_u['sexe'].value_counts())
print()
print(df_u['an_nais'].value_counts())
print()
print(df_u['trajet'].value_counts())
print()
print(df_u['secu'].value_counts())
print()
print(df_u['locp'].value_counts())
print()
print(df_u['actp'].value_counts())
print()
print(df_u['etatp'].value_counts())
print()


1.0    917361
2.0    128505
3.0     28353
4.0     24023
5.0     12418
7.0      3654
8.0      3640
9.0      3546
6.0      1237
Name: place, dtype: int64

1    917305
2    205432
3    107376
4      2120
Name: catu, dtype: int64

1    504629
4    442501
3    251731
2     33372
Name: grav, dtype: int64

1    831047
2    401186
Name: sexe, dtype: int64

1990.0    33560
1991.0    33005
1992.0    32383
1989.0    31776
1988.0    31259
          ...  
1912.0        5
1909.0        2
1908.0        2
1907.0        2
1906.0        2
Name: an_nais, Length: 114, dtype: int64

5.0    470736
0.0    324453
1.0    168745
4.0    119589
9.0     87166
3.0     33321
2.0     27826
Name: trajet, dtype: int64

11.0    700522
21.0    223718
13.0     91236
93.0     74863
23.0     22173
12.0     18600
22.0     13008
31.0      8774
3.0       7691
92.0      4931
91.0      4806
1.0       2729
2.0       2588
41.0      1703
42.0      1390
43.0      1091
33.0       464
32.0       419
Name: secu, dtype: int64

0.0    1

In [0]:
# Clean data and make all together

df_c_clean = df_c[['Num_Acc', 'an', 'lum', 'dep', 'agg', 'int', 'atm', 'col']].copy()
df_l_clean = df_l[['Num_Acc', 'catr', 'circ', 'nbv', 'vosp', 'plan', 'surf',
                   'infra', 'situ']].copy()
df_v_clean = df_v[['Num_Acc', 'num_veh', 'catv', 'obs', 'obsm', 'choc',
                   'manv']].copy()
df_total = df_u[['Num_Acc', 'num_veh','place', 'catu', 'grav', 'an_nais', 
                 'trajet', 'secu', 'locp', 'actp', 'etatp']].copy()

In [0]:
# we first join 'lieux' and 'caracteristiques' to 'usagers:
df_total = df_total.join(df_c_clean.set_index('Num_Acc'), on='Num_Acc')
df_total = df_total.join(df_l_clean.set_index('Num_Acc'), on='Num_Acc')

In [0]:
# Joining some informations together
# We use a temporary column to merge the informations from usagers
# and from vehicles
df_total.insert(1, 'temp', 'not determined')
df_v_clean.insert(1, 'temp', 'not determined')

# Vehicles
# simplify catv
sim = [[30,31,32,33,34], [7,10], [13, 14, 15, 16, 17, 20, 21, 37], [35, 36, 38, 39, 40, 99]]
clas = [2, 3, 4, 5]
for i in range(len(clas)):
  for j in sim[i]:
      df_v_clean = df_v_clean.replace({'catv': {j : clas[i]}})
print(df_v_clean['catv'].value_counts())  

3    639446
2    213565
1     42174
4     38463
5      8574
Name: catv, dtype: int64


In [0]:
# join together obs and obsm
for i in range(len(df_v_clean.index)):
  
  df_v_clean.at[i, 'temp'] = str(df_v_clean.at[i, 'Num_Acc']) + str(df_v_clean.at[i, 'num_veh'])

  if df_v_clean.at[i, 'obsm'] != 0:
    df_v_clean.at[i, 'obs'] = df_v_clean.at[i, 'obsm']
  else:
    if df_v_clean.at[i, 'obs'] != 0:
      df_v_clean.at[i, 'obs'] = 7 # fixed object
  
del df_v_clean['obsm']

# Usagers
# transforming an-nais in age
for i in range(len(df_total.index)):

  df_total.at[i, 'temp'] = str(df_total.at[i, 'Num_Acc']) + str(df_total.at[i, 'num_veh'])

  year = df_total.at[i, 'Num_Acc']
  year = int(str(year)[0:4])
  
  if not math.isnan(df_total.at[i, 'an_nais']):
    df_total.at[i, 'an_nais'] = int(year - df_total.at[i, 'an_nais'])

# Renames the column:
df_total.rename(columns={'an_nais' : 'age'}, inplace=True)

In [0]:
# Erase Num_acc and num_veh to make the join

del df_total['Num_Acc']
del df_total['num_veh']
del df_v_clean['Num_Acc']
del df_v_clean['num_veh']

df_total = df_total.join(df_v_clean.set_index('temp'), on='temp', how='left')

In [0]:
# Now we come back with number of accident and number of the vehicle

df_total.insert(0, 'num_acc', 0)
df_total.insert(1, 'num_veh', 'not determined')

for i in range(len(df_total.index)):
  value = df_total.at[i, 'temp']
  v_len = len(value)
  num_s = value[0 : v_len - 3]

  num = int(num_s)
  veh = value[v_len - 3 : v_len]

  df_total.at[i, 'num_acc'] = num
  df_total.at[i, 'num_veh'] = veh

In [0]:
# Delete the temp column

del df_total['temp']

In [0]:
# Transform NaN to means and etc

columns = list(df_total)
means = {'nbv'}
majority = {'place', 'trajet', 'agg', 'circ', 
            'vosp', 'plan', 'surf', 'infra', 'situ', 'atm', 'col', 'atm'}
zeros = {'locp', 'actp', 'etatp', 'obs', 'choc', 'manv'}

for column in columns:
  if column in means:
    df_total[column].fillna(value=df[column].mean(), inplace=True)
  if column in majority:
    df_total[column].fillna(value=df_total[column].value_counts().idxmax(), 
                            inplace=True)
  if column in zeros:
    df_total[column].fillna(value=0, inplace=True)
  if column == 'secu':
    df_total[column].fillna(value=3,inplace=True)
if column == 'age':
    df_total[column].fillna(-1, inplace=True)

df_total['age'].replace(to_replace=[-1], value=np.nan, inplace=True)
df_total['age'].fillna(df_total['age'].mean(), inplace=True)

In [0]:
df_total.info()
df_total.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232233 entries, 0 to 1232232
Data columns (total 30 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   num_acc  1232233 non-null  int64  
 1   num_veh  1232233 non-null  object 
 2   place    1232233 non-null  float64
 3   catu     1232233 non-null  int64  
 4   grav     1232233 non-null  int64  
 5   age      1232233 non-null  float64
 6   trajet   1232233 non-null  float64
 7   secu     1232233 non-null  float64
 8   locp     1232233 non-null  float64
 9   actp     1232233 non-null  float64
 10  etatp    1232233 non-null  float64
 11  an       1232233 non-null  int64  
 12  lum      1232233 non-null  int64  
 13  dep      1232233 non-null  int64  
 14  agg      1232233 non-null  int64  
 15  int      1232233 non-null  int64  
 16  atm      1232233 non-null  float64
 17  col      1232233 non-null  float64
 18  catr     1232233 non-null  int64  
 19  circ     1232233 non-null  float64
 20  nb

,num_acc,num_veh,place,catu,grav,age,trajet,secu,locp,actp,etatp,an,lum,dep,agg,int,atm,col,catr,circ,nbv,vosp,plan,surf,infra,situ,catv,obs,choc,manv
0,201800000001,B01,1.0,1,3,90.0,0.0,11.0,0.0,0.0,0.0,18,1,590,1,4,1.0,1.0,3,2.0,2.0,0.0,3.0,1.0,0.0,1.0,3,2.0,3.0,1.0
1,201800000001,A01,1.0,1,1,58.0,5.0,11.0,0.0,0.0,0.0,18,1,590,1,4,1.0,1.0,3,2.0,2.0,0.0,3.0,1.0,0.0,1.0,3,2.0,2.0,15.0
2,201800000002,A01,1.0,1,1,71.0,0.0,11.0,0.0,0.0,0.0,18,1,590,2,7,7.0,7.0,4,2.0,2.0,0.0,2.0,1.0,0.0,1.0,3,1.0,0.0,1.0
3,201800000002,A01,1.0,3,4,59.0,0.0,2.0,2.0,3.0,1.0,18,1,590,2,7,7.0,7.0,4,2.0,2.0,0.0,2.0,1.0,0.0,1.0,3,1.0,0.0,1.0
4,201800000003,A01,1.0,1,3,31.0,5.0,21.0,0.0,0.0,0.0,18,1,590,2,3,1.0,7.0,3,2.0,2.0,0.0,1.0,1.0,0.0,1.0,2,2.0,1.0,1.0


In [0]:
df_total.to_csv('dataframe.csv', index=False)

# Getting data and saving them at my Google Drive folder

In [0]:
# web scraping

!pip install selenium
!pip install beautifulsoup4
!pip install wget

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests
import wget
import traceback

     |████████████████████████████████| 911kB 7.9MB/s 
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=81cba61bf635e2cb329f8d77e60a61431ccdc0eb0fb62218056580a2017ec1d2
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
url = 'https://www.data.gouv.fr/fr/datasets/base-de-donnees-accidents-corporels-de-la-circulation/#_'

year = 2010 # First year

response = requests.get(url, timeout=10)
if response.status_code == 200:
  print('Permission to access granted')
  print()
soup = BeautifulSoup(response.content, 'html.parser')
# get list of names
list_names = []
# Selecting all article type
for file in soup.findAll('article', attrs={'class':'card resource-card '}):
  # Get the name of the file
  name = file.findAll('h4', attrs={'class':'ellipsis'})[0].get_text()
  # Check if its a file
  if name.endswith('.csv') and int(name[-8:-4]) >= year:
    # Get the file
    list_names.append(name)
    filename = file.findAll('a', attrs={'class':'btn btn-sm btn-primary'})[1]['href']
    filedata = wget.download(filename, 'sample_data/' + name)

Permission to access granted



In [0]:
v = []
u = []
l = []
c = []

def sort_names(name):
  return(name[:-10]+name[-8:])
list_names_s = sorted(list_names, key=sort_names)

n = len(list_names_s) / 4
for i, name in enumerate(list_names_s, 0):
  if 0 <= i < n:
    c.append(name)
  elif n <= i < 2*n:
    l.append(name)
  elif 2*n <= i < 3*n:
    u.append(name)
  else:
    v.append(name)

df_c = pd.DataFrame()
df_l = pd.DataFrame()
df_u = pd.DataFrame()
df_v = pd.DataFrame()

print("--------------------")
print("caracteristiques.csv")
print("--------------------")
er = []
for file in c:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_c = pd.concat([data, df_c], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_c))
    print("num_columns = %d" %len(df_c.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("lieux.csv")
print("--------------------")
er = []
for file in l:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_l = pd.concat([data, df_l], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_l))
    print("num_columns = %d" %len(df_l.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("usagers.csv")
print("--------------------")
er = []
for file in u:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_u = pd.concat([data, df_u], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_u))
    print("num_columns = %d" %len(df_u.columns))
print()
print('Error in years :')
print(er)
print()
print("--------------------")
print("vehicules.csv")
print("--------------------")
er = []
for file in v:
    try:
      data = pd.read_csv('sample_data/' + file, index_col='Num_Acc', engine='python')
      df_v = pd.concat([data, df_v], axis=0, sort=False)
    except Exception as e:
      traceback.print_exc()
      er.append(file[-8:-4])
    print()
    print("size = %d" %len(df_v))
    print("num_columns = %d" %len(df_v.columns))
print()
print('Error in years :')
print(er)
print()
df_c.to_csv('sample_data/caracteristiques.csv')
df_l.to_csv('sample_data/lieux.csv')
df_u.to_csv('sample_data/usagers.csv')
df_v.to_csv('sample_data/vehicules.csv')

--------------------
caracteristiques.csv
--------------------

size = 69379
num_columns = 15

size = 136353
num_columns = 15

size = 198603
num_columns = 15

size = 257000
num_columns = 15

size = 316854
num_columns = 15

size = 375508
num_columns = 15

size = 434940
num_columns = 15

size = 495641
num_columns = 15

size = 553424
num_columns = 15

Error in years :
[]

--------------------
lieux.csv
--------------------

size = 69379
num_columns = 17

size = 136353
num_columns = 17

size = 198603
num_columns = 17

size = 257000
num_columns = 17

size = 316854
num_columns = 17

size = 375508
num_columns = 17

size = 434940
num_columns = 17

size = 495641
num_columns = 17

size = 553424
num_columns = 17

Error in years :
[]

--------------------
usagers.csv
--------------------

size = 154192
num_columns = 11

size = 302735
num_columns = 11

size = 441363
num_columns = 11

size = 570057
num_columns = 11

size = 702243
num_columns = 11

size = 832621
num_columns = 11

size = 966043
num_co

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Pass data to my google drive
%mv "sample_data/lieux.csv" "drive/My Drive/Colab Notebooks/INF442_P8/lieux.csv"
%mv "sample_data/caracteristiques.csv" "drive/My Drive/Colab Notebooks/INF442_P8/caracteristiques.csv"
%mv "sample_data/usagers.csv" "drive/My Drive/Colab Notebooks/INF442_P8/usagers.csv"
%mv "sample_data/vehicules.csv" "drive/My Drive/Colab Notebooks/INF442_P8/vehicules.csv"

%cd "drive/My Drive/Colab Notebooks/INF442_P8/"

/content/drive/My Drive/Colab Notebooks/INF442_P8
